In [1]:
# How to improve long-term tendency and preserve information till the last stave.
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
from tqdm import tqdm


import warnings
warnings.filterwarnings('ignore')

import os
from matplotlib.colors import LinearSegmentedColormap

sns.set_theme(style="darkgrid")

np.random.seed(40)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(f"Device : {device}")


Device : cuda:0


In [2]:
#Functions

class SequenceDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, idx):
        x = self.data[idx:idx + self.seq_length]
        y = self.data[idx + self.seq_length]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)




# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True,dropout=0.3)
        self.fc = nn.Linear(hidden_size, output_size)


    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        
        
        out = self.fc(out[:, -1, :])
        return out


In [3]:
def model_training_and_evaluation(data, current_run_number_list):

    identifier = str(current_run_number_list)
    path = f'./Outputs/{identifier}'
    os.makedirs(path, exist_ok=True)

# --------------------------------------------------------------------------------------------------------------------------------------------------- #

    data = data.loc[data.RunNumber.isin(current_run_number_list)]
    data.RunNumber = data.RunNumber.astype('category')
    data = data[data.Sector.str.startswith('UTaX')]
    data = data[['RunNumber','Staves','Rows','PedestalValue']]
    pedestal = data.groupby(['RunNumber',  'Staves', 'Rows'], as_index=False)['PedestalValue'].mean()
    pedestal['PedestalValue'] = pedestal['PedestalValue'].fillna(method='ffill')



# --------------------------------------------------------------------------------------------------------------------------------------------------- #

    seq_length = 256

    train_data = pedestal[:-seq_length*2]
    test_data = pedestal[-seq_length*2:]

    train_data['PedestalValue'] = scaler.fit_transform(train_data[['PedestalValue']])
    test_data['PedestalValue'] = scaler.transform(test_data[['PedestalValue']])


    train_dataset = SequenceDataset(train_data['PedestalValue'].values, seq_length)
    test_dataset = SequenceDataset(test_data['PedestalValue'].values, seq_length)

    # Use DataLoader for batching
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


# --------------------------------------------------------------------------------------------------------------------------------------------------- #

    # Parameters
    input_size = 1
    hidden_size = 128
    num_layers = 2
    output_size = 1
    num_epochs = 1000

    # Initialize the model, loss function, and optimizer
    model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device=device)
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters())

# --------------------------------------------------------------------------------------------------------------------------------------------------- #

    # Training loop
    losses = []
    for epoch in tqdm(range(num_epochs)):
        model.train()
        epoch_loss = 0.0
        for x_batch, y_batch in train_loader:
            x_batch = x_batch.unsqueeze(-1).to(device)  # Add input_size dimension
            y_batch = y_batch.unsqueeze(-1).to(device)  # Ensure correct shape
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        losses.append(epoch_loss / len(train_loader))
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss / len(train_loader):.4f}')

# --------------------------------------------------------------------------------------------------------------------------------------------------- #
    # Loss Value Dumping

    losses_dataframe = pd.DataFrame(index=np.arange(num_epochs))
    losses_dataframe['LossValues'] = losses


    losses_dataframe.to_csv(f'{path}/loss.csv')
# --------------------------------------------------------------------------------------------------------------------------------------------------- #

    # Model Evaluation and Predictions

    model.eval()


    true_values = []
    predictions = []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.unsqueeze(-1).to(device)  # Add input_size dimension and move to GPU
            y_batch = y_batch.unsqueeze(-1).to(device) 
            preds = model(x_batch)

            # np.concatenate(preds.cpu().numpy())
            predictions.append(preds.cpu().numpy())  # Move predictions back to CPU for further processing
            true_values.append(y_batch.cpu().numpy())

    true_values = np.concatenate(true_values)
    predictions = np.concatenate(predictions)

    template = test_data.iloc[256:][['Staves','Rows']].reset_index(drop=True)

    analysis = template.copy()

    analysis['Sequence'] = test_data.iloc[:256]['PedestalValue'].values

    analysis['TrueValues'] = true_values
    analysis['Predictions'] = predictions

    analysis['Difference'] = analysis['TrueValues'] - analysis['Predictions']


    analysis.to_csv(f'{path}/staves_prediction.csv')

# --------------------------------------------------------------------------------------------------------------------------------------------------- #

    # Corresponding Training Data for the Forecasting Plot

    train_data.to_csv(f'{path}/train_sample.csv')

    return

In [4]:
# Read All Data and Fetching the RUn Numbers
data = pd.read_parquet('/home/hashmi/FileDrive/UpstreamTrackerCalibration/00Data/Inputs/FullData.parquet')
all_runs = sorted(data['RunNumber'].unique().tolist())

In [5]:
# Iterating through the Run Numbers as rolling window, and executing the function.
for i in range(10):
    current_run_number_list = all_runs[i:10+i]
    print(f"Current Runs : {current_run_number_list}")
    model_training_and_evaluation(data=data,current_run_number_list=current_run_number_list)
    print("Run Model Dump Completed")
    print("---")

Current Runs : [293528, 293530, 293532, 293533, 297288, 298593, 298594, 298595, 298596, 302935]


 10%|█         | 100/1000 [00:48<07:16,  2.06it/s]

Epoch [100/1000], Loss: 0.0257


 20%|██        | 200/1000 [01:36<06:26,  2.07it/s]

Epoch [200/1000], Loss: 0.0246


...

 90%|█████████ | 900/1000 [07:10<00:46,  2.17it/s]

Epoch [900/1000], Loss: 0.0097


100%|██████████| 1000/1000 [07:59<00:00,  2.09it/s]

Epoch [1000/1000], Loss: 0.0092


Run Model Dump Completed
---
